In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
SAVE_DIR = 'attribution/'
ATTR = 'SVS'
SAMPLES = 8000

In [2]:
from stable_baselines3 import PPO

import pandas as pd
import numpy as np

from captum.attr import ShapleyValueSampling
import torch

from citylearn.data import DataSet

import KBMproject.utilities as utils

In [3]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [4]:
actor = utils.extract_actor(agent)

In [5]:
SVS = ShapleyValueSampling(actor)

In [6]:
from art.estimators.classification import PyTorchClassifier as classifier
from torch.nn import CrossEntropyLoss
victim_policy = classifier(
    model=actor,
    loss=CrossEntropyLoss(), 
    nb_classes=agent.action_space[0].n,
    input_shape=agent.observation_space.shape,
    device_type='gpu',
    clip_values = (agent.observation_space.low.min(),agent.observation_space.high.max()) #min and max values of each feature, brendle bethge attack only supports floats values and not array
    )

In [7]:
df_clean_bb_obs = pd.read_csv('bb results/clean obs.csv',
                           index_col=0,
                           dtype='float32')
df_adv_bb_obs = pd.read_csv('bb results/bb obs.csv',
                         index_col=0,
                         dtype='float32')

In [8]:
clean_bb_a = np.argmax(victim_policy.predict(df_clean_bb_obs), axis=1)
adv_bb_a = np.argmax(victim_policy.predict(df_adv_bb_obs), axis=1)

In [9]:
df_clean_acg_obs = pd.read_csv('adv_obs.csv',
                           index_col=0,
                           dtype='float32')
df_adv_acg_obs = pd.read_csv('adv_perturbed_obs.csv',
                         index_col=0,
                         dtype='float32')

In [10]:
clean_acg_a = np.argmax(victim_policy.predict(df_clean_acg_obs), axis=1)
adv_acg_a = np.argmax(victim_policy.predict(df_adv_acg_obs), axis=1)

In [11]:
df_baseline_obs = pd.read_csv('baseline_obs.csv',
                         index_col=0,
                         dtype='float32')
baseline_a = np.loadtxt('baseline_obs_a.csv',
                         delimiter=',',
                         ).astype('int64')

In [12]:
rebaseline_obs = np.loadtxt('rebaseline obs.csv',
                         delimiter=',',
                         ).astype('float32')
rebaseline_a = np.loadtxt('rebaseline a.csv',
                         delimiter=',',
                         ).astype('int64')

In [13]:
clean_bb_shap = SVS.attribute(torch.from_numpy(
                                            df_clean_bb_obs.to_numpy()
                                            ).to('cuda'), #convert df to cuda tensor
                             target=torch.from_numpy(clean_bb_a.flatten()).to('cuda'), #convert actions to 1d list
                             n_samples=SAMPLES
                             ).detach().cpu().numpy() #move results to cpu

In [14]:
adv_bb_shap = SVS.attribute(torch.from_numpy(df_adv_bb_obs.to_numpy()).to('cuda'),
                             target=torch.from_numpy(adv_bb_a.flatten()).to('cuda'),
                             n_samples=SAMPLES
                             ).detach().cpu().numpy()

In [15]:
clean_acg_shap = SVS.attribute(torch.from_numpy(df_clean_acg_obs.to_numpy()).to('cuda'),
                             target=torch.from_numpy(clean_acg_a.flatten()).to('cuda'),
                             n_samples=SAMPLES
                             ).detach().cpu().numpy()

In [ ]:
adv_acg_shap = SVS.attribute(torch.from_numpy(df_adv_acg_obs.to_numpy()).to('cuda'),
                             target=torch.from_numpy(adv_acg_a.flatten()).to('cuda'),
                             n_samples=SAMPLES
                             ).detach().cpu().numpy()

In [ ]:
baseline_shap = SVS.attribute(torch.from_numpy(df_baseline_obs.to_numpy()).to('cuda'),
                             target=torch.from_numpy(baseline_a.flatten()).to('cuda'),
                             n_samples=SAMPLES
                             ).detach().cpu().numpy()

In [ ]:
rebaseline_shap = SVS.attribute(torch.from_numpy(rebaseline_obs).to('cuda'),
                             target=torch.from_numpy(rebaseline_a).to('cuda'),
                             n_samples=SAMPLES
                             ).detach().cpu().numpy()

In [ ]:
env = utils.make_discrete_env(schema=DataSet.get_schema(DATASET_NAME),  
                        action_bins=agent.action_space[0].n,
                        seed=42)
cols = env.observation_names                        

In [ ]:
np.savetxt(SAVE_DIR + f'clean bb {ATTR} {SAMPLES}.csv', clean_bb_shap, delimiter=',')
np.savetxt(SAVE_DIR + f'adv bb {ATTR} {SAMPLES}.csv', adv_bb_shap, delimiter=',')
np.savetxt(SAVE_DIR + f'clean acg {ATTR} {SAMPLES}.csv', clean_acg_shap, delimiter=',')
np.savetxt(SAVE_DIR + f'adv acg {ATTR} {SAMPLES}.csv', adv_acg_shap, delimiter=',')
np.savetxt(SAVE_DIR + f'baseline {ATTR} {SAMPLES}.csv', baseline_shap, delimiter=',')
np.savetxt(SAVE_DIR + f'rebaseline {ATTR} {SAMPLES}.csv', rebaseline_shap, delimiter=',')